In [ ]:
import pickle
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


load data from google drive: https://drive.google.com/drive/folders/1ufRGISCS4hOSuJjpJ36ydkvxrTZ64Nlp?usp=drive_link

In [ ]:
with open('/content/drive/MyDrive/paper/raw_data.pickle','rb') as r:
    raw_data = pickle.load(r)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datetime import date
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def parse(date_string):
    '''
      Change date into list of integers
    '''
    return [int(i) for i in date_string.split('-')]

def get_split_index(start_index,end_index,start_date,end_date):
    '''
    Calculate the start and end indices for a given date range.

    Args:
        start_index (str): The start date index in the format 'YYYY-MM-DD'.
        end_index (str): The end date index in the format 'YYYY-MM-DD'.
        start_date (str): The start date of the date range in the format 'YYYY-MM-DD'.
        end_date (str): The end date of the date range in the format 'YYYY-MM-DD'.

    Returns:
        tuple: A tuple containing two integers:
            - index_start (int): The number of days from the start_date to start_index.
            - index_end (int): The number of days from the start_date to end_index, plus one.
    '''
    ys,ms,ds = parse(start_index)
    ye,me,de = parse(end_index)
    ysd,msd,dsd = parse(start_date)
    yed,med,ded = parse(end_date)
    start_time = date(ys,ms,ds)
    end_time   = date(ye,me,de)
    start_time_date = date(ysd,msd,dsd)
    end_time_date   = date(yed,med,ded)
    index_start = (start_time - start_time_date).days
    index_end   = (end_time - start_time_date).days+1
    if index_start < 0 or (end_time - end_time_date).days > 0 or end_time==start_time:
        raise ValueError(f'index date invalid with condition {index_start} < 0 or {(end_time - end_time_date).days} > 0 or {end_time} == {start_time}')
    return index_start,index_end

template data for masking land data

In [ ]:
template_data = torch.isnan(torch.cat([torch.tensor(np.array(raw_data['center'])),
                                       torch.tensor(np.array(raw_data['north'])),
                                       torch.tensor(np.array(raw_data['south'])),
                                       torch.tensor(np.array(raw_data['west'])),
                                       torch.tensor(np.array(raw_data['east'])),
                                       torch.tensor(np.array(raw_data['u_geos'])),
                                       torch.tensor(np.array(raw_data['v_geos']))])).any(dim=0)

In [ ]:
class MLPDataset(Dataset):
    def __init__(self,
                 data,
                 start_index_date,
                 end_index_date,
                 plot = False,
                 number_out = 5,
                 number_in = 1,
                 mask = None,
                 start_date="2013-01-01",
                 end_date="2021-12-31"):
        self.number_out = number_out
        self.number_in = number_in
        self.plot = plot
        self.mask = mask
        self.start_index_date = start_index_date
        self.end_index_date   = end_index_date
        self.start_date = start_date
        self.end_date   = end_date
        self.start, self.end  = get_split_index(self.start_index_date,self.end_index_date,self.start_date,self.end_date)
        self.data = self.preprocess(data)

    def __len__(self):
        return len(self.data[0][1,:])

    def __getitem__(self, index):
        x,y = self.data[0][:,index],self.data[1][:,index]
        return x,y

    def clean(self, x,y):
        if self.mask is None:
            x_mask = torch.isnan(x[:,:17745]).any(dim=0)
            y_mask = torch.isnan(y[:,:17745]).any(dim=0)

            self.mask = torch.logical_or(x_mask,y_mask)
            # shape = int(x.shape[1]/17745)
            # self.mask = mask.expand(shape,17745).flatten()
        shape = int(x.shape[1]/17745)
        self.mask = self.mask.reshape(1,17745).expand(shape,17745).flatten()
        # Use masked_select to select non-NaN values along the desired axis
        filtered_x = x[:, ~self.mask]
        filtered_y = y[:, ~self.mask]
        return filtered_x, filtered_y

    def preprocess(self,data):
        # X
        # init x
        lat     = []
        lon     = []
        center  = []
        north   = []
        south   = []
        east    = []
        west    = []
        u_in    = []
        v_in    = []

        # iterate over days
        lat.append(torch.tensor(np.roll(data['lat'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],self.number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
        lon.append(torch.tensor(np.roll(data['lon'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],self.number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
        for number_in in range(self.number_in):
            center.append(torch.tensor(np.roll(data['center'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            north.append(torch.tensor(np.roll(data['north'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            south.append(torch.tensor(np.roll(data['south'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            east.append(torch.tensor(np.roll(data['east'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            west.append(torch.tensor(np.roll(data['west'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))

            u_in.append(torch.tensor(np.roll(data['u_geos'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            v_in.append(torch.tensor(np.roll(data['v_geos'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],number_in,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
        #join days
        lat     = torch.cat(lat)
        lon     = torch.cat(lon)
        center  = torch.cat(center)
        north   = torch.cat(north)
        south   = torch.cat(south)
        east    = torch.cat(east)
        west    = torch.cat(west)
        u_in    = torch.cat(u_in)
        v_in    = torch.cat(v_in)

        X = torch.cat([lat,lon,center,north,south,east,west,u_in,v_in])

        # Y
        # init y
        u_out = []
        v_out = []

        # iterate over days
        for number_out in range(1,1+self.number_out):
            u_out.append(torch.tensor(np.roll(data['u_geos'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],-number_out,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
            v_out.append(torch.tensor(np.roll(data['v_geos'][max(0,self.start-self.number_in+1):min(self.end+self.number_out,3287)],-number_out,axis=0), dtype=torch.float32)[self.number_in-1:-self.number_out,:,:].reshape(1,-1))
        #join days
        u_out = torch.cat(u_out)
        v_out = torch.cat(v_out)
        Y = torch.cat([u_out,v_out])

        if not self.plot:
            X,Y = self.clean(X,Y)
        return X,Y

In [ ]:
train_data = MLPDataset(raw_data,"2013-01-01","2018-12-31", number_out = 5, number_in = 7, mask = template_data)
test_data  = MLPDataset(raw_data,"2019-01-01","2021-12-31", number_out = 5, number_in = 7, mask = template_data)

In [ ]:
import torch
import torch.nn as nn

class DifferenceLoss(nn.Module):
    def __init__(self, epsilon=0.001):
        """
        Initialize the loss function.

        Args:
            alpha (float): The weight given to L1 loss. (1 - alpha) will be the weight for L2 loss.
        """
        super(DifferenceLoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, outputs, targets):
        """
        Calculate the combined L1 and L2 loss.

        Args:
            outputs (torch.Tensor): The predictions of the model.
            targets (torch.Tensor): The actual values.

        Returns:
            torch.Tensor: The calculated loss.
        """
        num_examples = outputs.size()[0]
        true = targets.reshape(num_examples,-1)
        pred = outputs.reshape(num_examples,-1)
        reconstruction_error= torch.sum(torch.pow((pred-true),2,),dim=1)

        true_diff = torch.diff(true.view(num_examples,2,-1)).view(num_examples,-1)
        pred_diff = torch.diff(pred.view(num_examples,2,-1)).view(num_examples,-1)
        difference_error    = torch.sum(torch.pow((pred_diff-true_diff),2),dim=1)

        combined_loss = torch.mean(difference_error+reconstruction_error)
        return combined_loss

from numpy import linalg as LA
def MSE(outputs, targets):
  """
  Calculate the combined L1 and L2 loss.

  Args:
    outputs (torch.Tensor): The predictions of the model.
    targets (torch.Tensor): The actual values.

    Returns:
    torch.Tensor: The calculated loss.
  """
  num_examples = outputs.shape[0]
  true = targets.reshape(num_examples,-1)
  pred = outputs.reshape(num_examples,-1)
  reconstruction_error = np.mean(np.power((pred-true),2))
  return reconstruction_error

In [ ]:
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=100,
    shuffle=True,
    #collate_fn=custom_collate_fn  # Pass your custom collate function if needed
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=100,
    shuffle=True,
    #collate_fn=custom_collate_fn  # Pass your custom collate function if needed
)

In [ ]:
class MLP(nn.Module):
    def __init__(self,num_input,num_output):
        super(MLP, self).__init__()
        # Define the first fully connected layer
        self.fc1 = nn.Linear(num_input, 100)  # Input dimension is 7, output dimension is 100
        # Define the first layer norm
        self.layer_norm1 = nn.LayerNorm(100)
        # Define the second fully connected layer
        self.fc2 = nn.Linear(100, 100)  # Input dimension is 100, output dimension is 100
        # Define the third fully connected layer
        self.fc3 = nn.Linear(100, 100)  # Input dimension is 100, output dimension is 100
        # Define the first layer norm
        self.layer_norm2 = nn.LayerNorm(100)
        # Define the output layer
        self.fc4 = nn.Linear(100, num_output)  # Input dimension is 100, output dimension is 10

    def forward(self, x):
        # Apply the first layer with a ReLU activation
        x = F.relu(self.fc1(x))
        # Apply the first norm layer
        x = self.layer_norm1(x)
        # Apply the second layer with a ReLU activation
        x = F.relu(self.fc2(x))
        # Apply the third layer with a ReLU activation
        x = F.relu(self.fc3(x))
        # Apply the second norm layer
        x = self.layer_norm2(x)
        # Apply the output layer
        x = self.fc4(x)
        return x

# Instantiate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP(51,10).to(device)

In [ ]:
import torch.optim as optim

# Define Adamax optimizer
optimizer = optim.Adamax(model.parameters(), lr=0.001)
# Define cross-entropy loss function
criterion = DifferenceLoss()#nn.MSELoss()

In [ ]:
num_epochs = 10
train_losses = []
test_losses = []
train_eval = []
test_eval = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_train_loss = 0.0
    epoch_train_eval = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_train_eval += MSE(outputs.cpu().detach().numpy(), labels.cpu().detach().numpy())*inputs.size(0)
        epoch_train_loss += loss.item()*inputs.size(0)
    epoch_train_eval /= len(train_dataloader.dataset)
    epoch_train_loss /= len(train_dataloader.dataset)
    train_losses.append(epoch_train_loss)
    train_eval.append(epoch_train_eval)
    # Evaluation phase
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_eval = 0.0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            epoch_test_eval += MSE(outputs.cpu().detach().numpy(), labels.cpu().detach().numpy())*inputs.size(0)  # Accumulate the loss for this batch
            epoch_test_loss += loss.item()*inputs.size(0)  # Accumulate the loss for this batch
    # Calculate average test loss for the epoch
    epoch_test_loss /= len(test_dataloader.dataset)
    epoch_test_eval /= len(test_dataloader.dataset)
    test_losses.append(epoch_test_loss)
    test_eval.append(epoch_test_eval)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: DiffE {epoch_train_loss:.4f}| MSE {epoch_train_eval:.4f}, Test Loss: DiffE {epoch_test_loss:.4f}| MSE {epoch_test_eval:.4f}')


Epoch [1/10], Train Loss: DiffE 0.1459| MSE 0.0118, Test Loss: DiffE 0.1435| MSE 0.0115
Epoch [2/10], Train Loss: DiffE 0.1397| MSE 0.0112, Test Loss: DiffE 0.1408| MSE 0.0113
Epoch [3/10], Train Loss: DiffE 0.1383| MSE 0.0111, Test Loss: DiffE 0.1409| MSE 0.0113
Epoch [4/10], Train Loss: DiffE 0.1374| MSE 0.0110, Test Loss: DiffE 0.1402| MSE 0.0112
Epoch [5/10], Train Loss: DiffE 0.1368| MSE 0.0109, Test Loss: DiffE 0.1399| MSE 0.0112
Epoch [6/10], Train Loss: DiffE 0.1363| MSE 0.0109, Test Loss: DiffE 0.1395| MSE 0.0112
Epoch [7/10], Train Loss: DiffE 0.1360| MSE 0.0109, Test Loss: DiffE 0.1414| MSE 0.0113
Epoch [8/10], Train Loss: DiffE 0.1356| MSE 0.0108, Test Loss: DiffE 0.1399| MSE 0.0112
Epoch [9/10], Train Loss: DiffE 0.1354| MSE 0.0108, Test Loss: DiffE 0.1393| MSE 0.0111
Epoch [10/10], Train Loss: DiffE 0.1351| MSE 0.0108, Test Loss: DiffE 0.1386| MSE 0.0111


In [ ]:
name = 'ANN_experiment4_ver1'
metadata = {'train_loss':train_losses,'test_loss':test_losses,'train eval':train_eval,'test eval':test_eval}
with open(f'metadata_{name}.pickle','wb') as m:
  pickle.dump(metadata,m,protocol=pickle.HIGHEST_PROTOCOL)
torch.save(model.state_dict(), f'model_{name}.pth')